In [1]:
import pandas as pd

In [5]:
test = pd.read_csv('notebooks/labs31_notebooks/train_data.csv')
test

,incident_id,src,categories,city,state,lat,long,title,desc,date,verbalization,empty_hand_soft,empty_hand_hard,less_lethal_methods,lethal_force,uncategorized
0,ny-newyorkcity-118,['https://twitter.com/_SNAKEDICK_/status/13546...,"['arrest', 'baton', 'push', 'shove', 'strike']",New York City,New York,40.664654,-73.948220,"Police charge, assault, and arrest protesters",At a protest outside the 71st precinct in Broo...,2021-01-26 00:00:00+00:00,False,True,False,False,False,False
1,or-portland-422,['https://twitter.com/gravemorgan/status/13532...,"['arrest', 'less-lethal', 'pepper-ball', 'proj...",Portland,Oregon,45.493156,-122.672540,Federal officers push protesters from ICE faci...,In an attempt to make targeted arrests and pus...,2021-01-23 00:00:00+00:00,False,True,False,False,False,False
2,or-portland-423,['https://twitter.com/_WhatRiot/status/1353434...,"['less-lethal', 'pepper-ball', 'projectile', '...",Portland,Oregon,45.493156,-122.672540,Federal agents injure then shoot activist,After being struck by a stun grenade that alle...,2021-01-23 00:00:00+00:00,False,False,False,True,False,False
3,or-portland-420,['https://twitter.com/gravemorgan/status/13521...,"['less-lethal', 'pepper-ball', 'pepper-spray',...",Portland,Oregon,45.493980,-122.671870,Police pepper spray protesters walking away,During a bullrush down Moody from the ICE faci...,2021-01-20 00:00:00+00:00,False,False,False,True,False,False
4,or-portland-419,['https://twitter.com/gravemorgan/status/13521...,"['abuse-of-power', 'arrest', 'less-lethal', 'p...",Portland,Oregon,45.493492,-122.671980,Police pepper spray detained protester,After pushing protesters from the ICE facility...,2021-01-20 00:00:00+00:00,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,ny-newyorkcity-55,['https://twitter.com/ShimonPro/status/1266136...,"['arrest', 'baton', 'grab', 'shove', 'strike']",New York City,New York,40.736550,-73.991140,"Police make violent arrests, officer breaks ba...",Footage taken at Union Square and East 17th st...,2020-05-28 00:00:00+00:00,False,True,False,False,False,False
1286,co-denver-1,['https://www.denverpost.com/2020/05/29/denver...,"['less-lethal', 'pepper-ball', 'shoot']",Denver,Colorado,39.738440,-104.986260,Reporter shot with multiple pepper balls,"He states the officer aimed at him, and the pe...",2020-05-28 00:00:00+00:00,False,False,False,True,False,False
1287,mn-minneapolis-21,"['https://youtu.be/XAa5xb6JitI?t=5982', 'https...","['baton', 'beat', 'strike']",Minneapolis,Minnesota,44.947865,-93.234886,Police hit press in neck and head with wooden ...,A group of cops start to approach a group of p...,2020-05-26 00:00:00+00:00,False,False,False,True,False,False
1288,mn-minneapolis-28,['https://www.facebook.com/damicedsota.thespir...,"['abuse-of-power', 'arrest']",Minneapolis,Minnesota,44.941326,-93.262610,Man has his gun confiscated in an open carry s...,Man encounters police arresting people open ca...,2020-05-26 00:00:00+00:00,False,False,False,False,False,True
